### 1. 앙상블 ① - 이미지 학습모델 3개

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:10<00:00, 931635.91it/s] 


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 132680.49it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1257750.61it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2559522.88it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [2]:
from torchvision.models import resnet18

# ResNet18
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.model = resnet18(pretrained=False)
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.fc = nn.Linear(self.model.fc.in_features, 10)

    def forward(self, x):
        return self.model(x)

resnet_model = ResNetModel().to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
from torchvision.models import densenet121

# DenseNet121 모델
class DenseNetModel(nn.Module):
    def __init__(self):
        super(DenseNetModel, self).__init__()
        self.model = densenet121(pretrained=False)
        self.model.features.conv0 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, 10)

    def forward(self, x):
        return self.model(x)

densenet_model = DenseNetModel().to(device)


In [4]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.MaxPool2d(2, 2)(x)
        x = nn.ReLU()(self.conv2(x))
        x = nn.MaxPool2d(2, 2)(x)
        x = x.view(-1, 64 * 56 * 56)
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

cnn_model = CNNModel().to(device)


In [6]:
import torch.optim as optim
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()

def train_model(model, optimizer, train_loader, epochs=1):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

resnet_optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)
densenet_optimizer = optim.Adam(densenet_model.parameters(), lr=0.001)
cnn_optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# 순차적으로 학습.
# ResNet
train_model(resnet_model, resnet_optimizer, train_loader)

# DenseNet
train_model(densenet_model, densenet_optimizer, train_loader)

# CNN
train_model(cnn_model, cnn_optimizer, train_loader)


100%|██████████| 938/938 [03:27<00:00,  4.52it/s]


Epoch 1, Loss: 0.026949115612626404


100%|██████████| 938/938 [10:28<00:00,  1.49it/s]


Epoch 1, Loss: 0.05763620306964346


100%|██████████| 938/938 [02:00<00:00,  7.77it/s]

Epoch 1, Loss: 0.20287329705704107


In [7]:
# 3개 모델 준비.
models = [resnet_model, densenet_model, cnn_model]

correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in tqdm(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = [model(inputs) for model in models]
        outputs = torch.mean(torch.stack(outputs), dim=0)

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Ensemble Accuracy: {100 * correct / total:.2f}%')

100%|██████████| 157/157 [00:52<00:00,  3.00it/s]

Ensemble Accuracy: 99.48%


### 실습1) 와인데이터를 통한 MLP 모델 3개 클래스 분류 앙상블

각 모델을 튜닝하여 성능을 높여보자.

In [144]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from torch.utils.data import DataLoader, TensorDataset

data = pd.read_csv('wine.csv')

In [173]:
X = data.drop(columns=['class']).values
y = data[['class']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [174]:
import torch.nn as nn

class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.fc1 = nn.Linear(12, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = self.fc3(x)
        return x

mlp_model1 = Model1()


In [175]:
class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.fc1 = nn.Linear(12, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = self.fc3(x)
        return x

mlp_model2 = Model2()


In [176]:
class Model3(nn.Module):
    def __init__(self):
        super(Model3, self).__init__()
        self.fc1 = nn.Linear(12, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = self.fc3(x)
        return x

mlp_model3 = Model3()


In [177]:
import torch.optim as optim

#criterion = nn.CrossEntropyLoss()
criterion = nn.BCEWithLogitsLoss()

optimizer1 = optim.SGD(mlp_model1.parameters(), lr=0.01)
optimizer2 = optim.SGD(mlp_model2.parameters(), lr=0.01)
optimizer3 = optim.SGD(mlp_model3.parameters(), lr=0.01)


In [178]:
epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer1.zero_grad()
        outputs = mlp_model1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer1.step()
        running_loss += loss.item()
    print(f"MLP Model 1, Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


MLP Model 1, Epoch 1, Loss: 0.6840338023697458
MLP Model 1, Epoch 2, Loss: 0.626703308122914
MLP Model 1, Epoch 3, Loss: 0.5803060553422789
MLP Model 1, Epoch 4, Loss: 0.5371993042346908
MLP Model 1, Epoch 5, Loss: 0.4905680530681843


In [179]:
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer2.zero_grad()
        outputs = mlp_model2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer2.step()
        running_loss += loss.item()
    print(f"MLP Model 2, Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


MLP Model 2, Epoch 1, Loss: 0.620295582748041
MLP Model 2, Epoch 2, Loss: 0.5679339926417281
MLP Model 2, Epoch 3, Loss: 0.5196893607697836
MLP Model 2, Epoch 4, Loss: 0.46562711676446406
MLP Model 2, Epoch 5, Loss: 0.3996948857859867


In [180]:
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer3.zero_grad()
        outputs = mlp_model3(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer3.step()
        running_loss += loss.item()
    print(f"MLP Model 3, Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


MLP Model 3, Epoch 1, Loss: 0.6459484609161935
MLP Model 3, Epoch 2, Loss: 0.5646071041502604
MLP Model 3, Epoch 3, Loss: 0.4917079352024125
MLP Model 3, Epoch 4, Loss: 0.41560937137138554
MLP Model 3, Epoch 5, Loss: 0.33834616385582017


In [181]:
import torch.nn.functional as F

models = [mlp_model1, mlp_model2, mlp_model3]

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = [model(inputs) for model in models]
        outputs = torch.mean(torch.stack(outputs), dim=0)
        outputs = torch.sigmoid(outputs)
        predicted = (outputs >= 0.5).long().squeeze()
        labels = labels.view_as(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Ensemble Accuracy: {accuracy:.2f}%')


Ensemble Accuracy: 91.38%


### 실습2) 와인데이터를 통한 MLP 모델 3개 퀄리티 분류 앙상블

각 모델을 튜닝하여 성능을 높여보자.

In [183]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from torch.utils.data import DataLoader, TensorDataset

data = pd.read_csv('wine.csv')

data['quality'] = LabelEncoder().fit_transform(data['quality'])

In [198]:
X = data.drop(columns=['quality']).values
y = data['quality'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [199]:
import torch.nn as nn

class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.fc1 = nn.Linear(12, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 7)

    def forward(self, x):
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = self.fc3(x)
        return x

mlp_model1 = Model1()


In [200]:
class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.fc1 = nn.Linear(12, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 7)

    def forward(self, x):
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = self.fc3(x)
        return x

mlp_model2 = Model2()


In [201]:
class Model3(nn.Module):
    def __init__(self):
        super(Model3, self).__init__()
        self.fc1 = nn.Linear(12, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 7)

    def forward(self, x):
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = self.fc3(x)
        return x

mlp_model3 = Model3()


In [202]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

optimizer1 = optim.SGD(mlp_model1.parameters(), lr=0.01)
optimizer2 = optim.SGD(mlp_model2.parameters(), lr=0.01)
optimizer3 = optim.SGD(mlp_model3.parameters(), lr=0.01)


In [203]:
epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer1.zero_grad()
        outputs = mlp_model1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer1.step()
        running_loss += loss.item()
    print(f"MLP Model 1, Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


MLP Model 1, Epoch 1, Loss: 1.8964572912309228
MLP Model 1, Epoch 2, Loss: 1.6932221607464115
MLP Model 1, Epoch 3, Loss: 1.5409346586320458
MLP Model 1, Epoch 4, Loss: 1.4328416309705594
MLP Model 1, Epoch 5, Loss: 1.368042553343424


In [204]:
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer2.zero_grad()
        outputs = mlp_model2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer2.step()
        running_loss += loss.item()
    print(f"MLP Model 2, Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


MLP Model 2, Epoch 1, Loss: 1.8909091208039261
MLP Model 2, Epoch 2, Loss: 1.6908259028341712
MLP Model 2, Epoch 3, Loss: 1.5310511385522239
MLP Model 2, Epoch 4, Loss: 1.427418538709966
MLP Model 2, Epoch 5, Loss: 1.3688525063235586


In [205]:
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer3.zero_grad()
        outputs = mlp_model3(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer3.step()
        running_loss += loss.item()
    print(f"MLP Model 3, Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


MLP Model 3, Epoch 1, Loss: 1.88933570501281
MLP Model 3, Epoch 2, Loss: 1.681196342154247
MLP Model 3, Epoch 3, Loss: 1.525530820939599
MLP Model 3, Epoch 4, Loss: 1.42494510732046
MLP Model 3, Epoch 5, Loss: 1.3616052618840846


In [206]:
models = [mlp_model1, mlp_model2, mlp_model3]

correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = [model(inputs) for model in models]
        outputs = torch.mean(torch.stack(outputs), dim=0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total
print(f'Ensemble Accuracy: {accuracy:.2f}%')

Ensemble Accuracy: 45.92%


### 2. 멀티모달 학습

1. MNIST 이미지와 벡터화된 MNIST 이미지를 CNN과 MLP로 특징 추출
2. 그 후, 두가지 피처벡터를 합친다음에 FC로 최종 아웃풋 내는 모델에 전달

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

batch_size = 64
learning_rate = 0.001
num_epochs = 10

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 7 * 7)
        x = F.relu(self.fc1(x))
        return x

# MLP
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return x

# 멀티모달모델
class MultimodalModel(nn.Module):
    def __init__(self, cnn_model, mlp_model, combined_hidden_size, num_classes):
        super(MultimodalModel, self).__init__()
        self.cnn = cnn_model
        self.mlp = mlp_model
        self.fc_combined = nn.Linear(128 + 128, combined_hidden_size)
        self.fc_out = nn.Linear(combined_hidden_size, num_classes)

    def forward(self, image, flat_image):
        image_features = self.cnn(image)
        flat_image_features = self.mlp(flat_image)
        combined = torch.cat((image_features, flat_image_features), dim=1)
        x = F.relu(self.fc_combined(combined))
        x = self.fc_out(x)
        return x

cnn_model = CNN()
mlp_model = MLP(input_size=784, hidden_size=128)  # 28x28 이미지를 펼친 후 784차원 입력
multimodal_model = MultimodalModel(cnn_model, mlp_model, combined_hidden_size=128, num_classes=10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(multimodal_model.parameters(), lr=learning_rate)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
multimodal_model = multimodal_model.to(device)

for epoch in range(num_epochs):
    multimodal_model.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # 이미지를 펼쳐서 정형 데이터로 변환
        flat_images = images.view(images.size(0), -1).to(device)

        optimizer.zero_grad()

        outputs = multimodal_model(images, flat_images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')


    multimodal_model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)

            # 이미지를 펼쳐서 정형 데이터로 변환
            flat_images = images.view(images.size(0), -1).to(device)

            outputs = multimodal_model(images, flat_images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')


100%|██████████| 938/938 [00:48<00:00, 19.15it/s]


Epoch [1/10], Loss: 0.1936


100%|██████████| 157/157 [00:05<00:00, 29.87it/s]


Validation Accuracy: 98.26%


100%|██████████| 938/938 [00:52<00:00, 17.91it/s]


...

Validation Accuracy: 99.07%


100%|██████████| 938/938 [00:48<00:00, 19.36it/s]


Epoch [10/10], Loss: 0.0095


100%|██████████| 157/157 [00:04<00:00, 33.93it/s]

Validation Accuracy: 99.15%
